In [77]:
import os
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from sklearn.model_selection import train_test_split

from torch.utils import data
import pandas as pd

In [78]:
import numpy as np
from scipy.io import loadmat

In [79]:
data_points = os.listdir("./Training_WFDB/")

In [80]:
data_points = set([i[:-4] for i in data_points])

In [81]:
data_points = list(data_points)

In [111]:
label_types = {
    "AF" : 0,
    "I-AVB" : 1,
    "LBBB" : 2,
    "Normal" : 3,
    "PAC" : 4,
    "PVC" : 5,
    "RBBB" : 6,
    "STD" : 7,
    "STE" : 8
} 

In [82]:
labels = []
for i in data_points:
    with open("./Training_WFDB/" + i + ".hea", "r") as f:
        file = f.readlines()
    for line in file:
        if line.startswith("#Dx"):
            label = line.split(":")[1].strip()
            labels.append(label)

In [91]:
X_train, X_test, y_train, y_test = train_test_split(data_points, labels, test_size=0.2, shuffle=True)

In [128]:
class DataSet(data.Dataset):
    def __init__(self, input_x, output_y):
        self.inputs = input_x
        self.outputs = output_y
        
    def __len__(self):
        return len(self.inputs)
    
    def __getitem__(self, idx):
        file = self.inputs[idx]
        data = loadmat("./Training_WFDB/" + file + ".mat")["val"]
        data = np.pad(data, ((0, 0), (0, 72000 - data.shape[1])), 'constant', constant_values=(0))
        return torch.Tensor(data.T), self.outputs[idx]

In [129]:
dataset = DataSet(X_train, y_train)
dataloader = data.DataLoader(dataset, batch_size=32, shuffle=True)

In [189]:
class Model(nn.Module):
    def __init__(self):
        super(Model, self).__init__()
        self.layer1 = nn.Linear(12, 1)
        self.convLayer1 = nn.Conv1d(in_channels=1, out_channels=3, kernel_size=5)
        self.maxpool1 = nn.MaxPool1d(kernel_size=5)
        self.convlayer2 = nn.Conv1d(in_channels=3, out_channels=5, kernel_size=5)
        self.maxpool2 = nn.MaxPool1d(kernel_size=5)
        self.convlayer3 = nn.Conv1d(in_channels=5, out_channels=7, kernel_size=5)
        self.maxpool3 = nn.MaxPool1d(kernel_size=5)
        self.layer2 = nn.Linear(7 * 575, 256)
        self.layer3 = nn.Linear(256, 32)
        self.layer4 = nn.Linear(32, 9)
        
        
    def forward(self, inp):
        inp = self.layer1(inp)
        inp = inp.transpose(1, 2)
        inp = torch.relu(self.convLayer1(inp))
        inp = self.maxpool1(inp)
        inp = torch.relu(self.convlayer2(inp))
        inp = self.maxpool2(inp)
        inp = torch.relu(self.convlayer3(inp))
        inp = self.maxpool3(inp)
        inp = inp.view(-1, 7 * inp.shape[2])
        inp = torch.relu(self.layer2(inp))
        inp = torch.relu(self.layer3(inp))
        inp = self.layer4(inp)
        
        return inp